### Project Code

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import networkx as nx
from math import *
import random 
from numpy import linalg as LA
from sklearn.datasets import fetch_openml
from sklearn.metrics import confusion_matrix, precision_score, recall_score,f1_score
import copy
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [2]:
mnist = fetch_openml('mnist_784', version=1)
U0, v0 = mnist["data"], mnist["target"]
U= U0.astype(np.double)
v = v0.astype(np.uint8)

In [3]:
v_bin_5_lst = [2*int(v[i]==5)-1 for i in range(len(v))]

In [4]:
df_U = pd.DataFrame(data=U)
df_v = pd.DataFrame(data=np.asarray(v_bin_5_lst),  columns=['label'])
df_data_merged =pd.concat([df_U, df_v.reindex(df_U.index)], axis=1)
df_data_merged.head()

,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,label
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1


In [5]:
def split_train_test(df_data_merged, train_set_size,test_set_size,m):
    np.random.seed(0)
    shuffled_indices = np.random.permutation(len(df_data_merged))
    batch_size = int(train_set_size/m)
    dic_train_sets_indices= {}
    dic_train_sets = {}
    for i in range(m):
        dic_train_sets_indices[i] = shuffled_indices[i*batch_size:(i+1)*batch_size]
        dic_train_sets[i] = df_data_merged.iloc[dic_train_sets_indices[i]]
    dic_train_set = {}
    dic_train_set_indices = shuffled_indices[:m*batch_size]
    dic_train_set[0] = df_data_merged.iloc[dic_train_set_indices]
    dic_test_set= {}
    test_indices = shuffled_indices[-test_set_size:]
    dic_test_set[0] = df_data_merged.iloc[test_indices]
    return dic_train_set, dic_train_sets, dic_test_set

In [6]:
def f_obj_global(x,dic_train,mu_param,m):
    output = sum([f_obj_local(x,dic_train[i],mu_param,m) for i in range(m)])
    return output

def f_obj_local(y,df_data,mu_param,m):
    npar_data= (df_data).to_numpy()
    nparr_data_transp = npar_data.T
    data = nparr_data_transp[:-1,:]
    labels =nparr_data_transp[-1:,:]
    n_attributes,n_labels = np.shape(data)
    x=y.reshape((n_attributes,1))
    v1 = np.dot(data.T,x) 
    v2 = -np.multiply(labels.T,v1)
    obj_val = 0
    obj_val = sum([v2[i,0] if v2[i,0] > 709 else log(1+exp(v2[i,0])) for i in range(n_labels)])
    output = obj_val + mu_param*np.dot(x.T,x)/(2*m)
    return output

def grad_f_local(x,df_data,mu_param,m):
    npar_data= (df_data).to_numpy()
    nparr_data_transp = npar_data.T
    data = nparr_data_transp[:-1,:]
    labels =nparr_data_transp[-1:,:]
    n_attributes,n_labels = np.shape(data)
    output = np.zeros((n_attributes,1))
    output = sum(-(labels[0,i]/(1+exp(min(709,labels[0,i]*np.dot(data[:,i],x)))))*data[:,[i]] for i in range(n_labels)) 
    return output+mu_param*x/m

def grad_f_matrix(x_matrix,dic_train_sets,mu_param):
    (m,n) = np.shape(x_matrix)
    output = np.zeros((m,n))
    for i in range(m):
        x = x_matrix[i,:].reshape((n,1))
        grad_vec = grad_f_local(x,dic_train_sets[i],mu_param,m)
        output[i,:] = grad_vec.reshape((1,n))
    return output

In [7]:
def R_matrix(graph_name,m):
    output = np.zeros((m,m))
    if m==1:
        output =np.array([[1]])
    elif graph_name == "star":
        for i in range(m):
            output[i,i]+=0.5
            output[i,0]+=0.5
    elif graph_name == "ring":
        for i in range(m):
            output[i,i]=0.5
            output[np.remainder(i+1,m) ,i]=0.5
    elif graph_name == "line":
        for i in range(m-1):
            output[i,i]=0.5
            output[i+1 ,i]=0.5
        output[0,0]=1
        output[m-1,m-1]=0.5
        output[2,1]=0
        output[2,2]=1
    elif graph_name == "complete":
        output = np.ones((m,m))/(2*(m-1))
        for i in range(m):
            output[i,i]=0.5
        #output = np.ones((m,m))/m 
    else:
        print("The graph name is unknown!")
    return output

def C_matrix(graph_name,m):
    output = R_matrix(graph_name,m).T
    return output

In [8]:
def gradient(x,data2,labels,mu_param):
    data = data2
    n_attributes,n_labels = np.shape(data)
    output = sum(-(labels[i]/(1+exp(min(709,labels[i]*np.dot(data[:,i],x)))))*data[:,i] 
                for i in range(n_labels))/n_labels
    return output+mu_param*x

def obj_function(x,data1,labels,mu_param):
    data=data1
    n_attributes,n_labels = np.shape(data)
    obj_val = 0
    v1 = np.dot(data.T,x) 
    v2 = -np.multiply(labels.T,v1)
    for i in range(n_labels):
        if v2[i] > 709:
            obj_val += v2[i]
        else:
            obj_val += log(1+exp(v2[i]))
    output = np.asscalar((obj_val/n_labels) + (mu_param*np.dot(x.T,x)/2))
    return output

In [9]:
def pushpull(dic_train_sets,max_iter,epoch_size,x0,G_R_name,G_C_name,step_size,mu_param):
    (m,n) = np.shape(x0)
    R = R_matrix(G_R_name,m)
    C = C_matrix(G_C_name,m)    
    f_vals = np.zeros(epoch_size+1)
    consensus_err_vals = np.zeros(epoch_size+1)
    x_matrix_now = x0
    x_matrix_next = x0
    grad_f_matrix_now = grad_f_matrix(x_matrix_now,dic_train_sets,mu_param)
    grad_f_matrix_next = grad_f_matrix(x_matrix_next,dic_train_sets,mu_param) 
    epoch_index = 0
    for k in range(max_iter+1):
        x_matrix_now = x_matrix_next
        grad_f_matrix_now = grad_f_matrix_next
        x_matrix_next = np.dot(R,x_matrix_now - np.dot(np.diag(step_size[0]),grad_f_matrix_now))
        grad_f_matrix_next = np.dot(C,grad_f_matrix_now)+grad_f_matrix(x_matrix_next,dic_train_sets,mu_param)-grad_f_matrix(x_matrix_now,dic_train_sets,mu_param)
        if max_iter==0 or (k % ceil(max_iter/epoch_size)) == 0:
            if m == 1:
                x_ave = x_matrix_now[0]
            else:
                x_ave = x_matrix_now.mean(0)
            f_vals[epoch_index] = f_obj_global(x_ave,dic_train_sets,mu_param,m)
            consensus_err_vals[epoch_index] = LA.norm(x_matrix_now-np.dot(np.ones((m,1)),x_ave.reshape(1,n)))
            epoch_index += 1
    return x_matrix_next, f_vals, consensus_err_vals 


def GDM(dic_train_set,max_iter,epoch_size,x0_GDM,step_size,mu_param):
    f_values = np.zeros(epoch_size+1)
    (m,n) = np.shape(x0_GDM)
    x_now = x0_GDM
    x_next = x0_GDM
    e=1
    consensus_err_vals = np.zeros(epoch_size+1)
    epoch_index = 0
    for k in range(max_iter+1):
        x_now = x_next
        x_next = x_now - step_size*grad_f_local(x_now,dic_train_set[0],mu_param,1)
        if max_iter==0 or (k % ceil(max_iter/epoch_size)) == 0:
            if m == 1:
                x_ave = x_now[0]
            else:
                x_ave = x_now.mean(0)
            f_values[epoch_index] = f_obj_global(x_now,dic_train_set,mu_param,1)
            consensus_err_vals[epoch_index] = LA.norm(x_now-np.dot(np.ones((m,1)),x_ave.reshape(1,n)))
            epoch_index += 1
    return x_next, f_values,consensus_err_vals 

def GD_by_pushpull(dic_train_set,max_iter,epoch_size,x0,step_size,mu_param):
    G_R_name = "complete"
    G_C_name = "complete"
    return pushpull(dic_train_set,max_iter,epoch_size,x0,G_R_name,G_C_name,step_size,mu_param)

In [10]:
def PIGM(data,max_iter,epoch,stepsize,mu_param,m,x0_PIGM):
    obj_values = np.zeros((epoch+1,m))
    glob_obj_values = np.zeros((epoch+1))
    n_attributes,n_labels = np.shape(data[0].values[:,:-1].T)
    x_now = np.ones((n_attributes,m+1))
    x_next = np.ones((n_attributes,m+1))
    consensus_err_vals = np.zeros(epoch_size+1)
    x_bar = x_next[:,:-1]
    a=1
    initial_x = np.ones((n_attributes,1))
    extra = initial_x
    epoch_index = 0
    epoch_index1 = 0
    for k in range(max_iter):
        x_now = x_next
        x_next[:,0] = x_now[:,m]
        for l in range(m):
            vector =  x_next[:,l] - stepsize*gradient(x_next[:,l],data[l].values[:,:-1].T ,data[l].values[:,-1].T,mu_param)
            #print(vector)
            #print(vector.shape)
            for i in range(n_attributes):
                x_next[i,l+1] = min(max(-1000,vector[0][i]),1000)
            #    if vector[0][i]>-1000:
            #        dummy = vector[0][i]
            #    else:
            #        dummy = -1000
            #    if(dummy<1000):
            #        dummy = dummy
            #    else:
            #        dummy = 1000
            #    x_next[i,l+1] = dummy 
        if k==0:
            x_bar = x_next.T[:-1][0]
        else:
            x_bar = x_next.T[:-1].mean(0)
        if (k % ceil(max_iter/epoch)) == 0:
            for count in range(m):
                obj_values[epoch_index,count] = obj_function(x_next[:,count],data[count].values[:,:-1].T,data[count].values[:,-1].T,mu_param)
            glob_obj_values[epoch_index] = sum([obj_values[epoch_index,j] for j in range(m)])
            consensus_err_vals[epoch_index] = LA.norm(x_next.T[:-1]-np.dot(np.ones((m,1)),x_bar.reshape(1,n_attributes)))
            epoch_index += 1
        glob_obj_values[epoch] =glob_obj_values[epoch-1]
        consensus_err_vals[epoch] = consensus_err_vals[epoch-1]
    return x_bar.reshape((1,n_attributes)), glob_obj_values,consensus_err_vals

In [11]:
def phi_func(e):
    if e > 709:
        dummy = 1
    elif e < -709:
        dummy = -1
    else:
        dummy = (exp(e)-exp(-e))/(exp(e)+exp(-e))
    return dummy

def f_grad(x, u, v):
    a = np.insert(u, 0, 1)
    a.reshape(len(a), 1)
    grad = (phi_func(a.T@x)-v)*(1-(phi_func(a.T@x))**2)*a
    return grad

def h_func(x, u):
    h=phi_func(x[0]+x[1]*u)
    return h

def obj_function(x,u,v):
    n_attributes, n_labels = u.shape
    total = 0
    for i in range(n_labels):
        total += ((h_func(x,u[0,i])-v[0,i])**2)/2
    return total/n_labels
       
def SGD(train_set_size, data, labels, max_iter, epoch_size, sample, x_init, step_size):
    f_values=np.zeros([epoch_size+1,sample])
    for j in range(sample):
        x_now = x_init 
        x_next = x_init 
        epoch_index = 0
        for k in range(max_iter+1):
            i = np.random.randint(train_set_size) 
            n_attributes,n_labels = np.shape(data)
            u = data[:, i]
            v = labels[0, i]
            x_next=x_now-step_size*f_grad(x_now,u,v)
            x_now=x_next
            if max_iter==0 or (k % ceil(max_iter/epoch_size))==0:
                f_values[epoch_index,j] = obj_function(x_now,data,labels)
                epoch_index += 1
    return f_values

In [12]:
def my_confu_mat(dic_test_set,opt_sol):
    npar_data= (dic_test_set[0]).to_numpy()
    test_data = npar_data[:,:-1]
    test_labels =npar_data[:,-1:]
    test_set_size = len(dic_test_set[0])
    pred_labels = np.zeros((test_set_size,1))
    for j in range(test_set_size):
        pred_labels[j][0] = np.sign(np.dot(test_data[j,:],opt_sol))
    output = confusion_matrix(test_labels, pred_labels)
    return output

def precision_score(dic_test_set,opt_sol):
    confu_mat = my_confu_mat(dic_test_set,opt_sol)
    output = confu_mat[1][1]/(confu_mat[1][0]+confu_mat[1][1])
    return output

In [29]:
n_attributes = 1
n_labels = 70000
np.random.seed(1)
mu, sigma = 10, 100
rnd_data = np.random.normal(mu, sigma, size=[n_labels, n_attributes])
#rnd_labels = 2*np.reshape(np.random.randint(2, size=n_labels),(n_labels,1)) - np.ones((n_labels,1))

df_rnd_data = pd.DataFrame(data=rnd_data )
df_rnd_labels = pd.DataFrame(data=np.asarray(v_bin_5_lst),  columns=['label'])
df_data_merged_rnd =pd.concat([df_rnd_data, df_rnd_labels.reindex(df_rnd_data.index)], axis=1)
# df_data_merged_rnd.head()

In [30]:
train_set_size, test_set_size, m = 1000, 1000, 5
dic_train_set, dic_train_sets, dic_test_set = split_train_test(df_data_merged_rnd, train_set_size, test_set_size, m)
df = dic_train_sets[0]
dataset= (df).to_numpy()
data_new = dataset.T
data = data_new[:-1,:]
labels =data_new[-1:,:]

In [31]:
n_attributes = len(df_data_merged_rnd.columns)-1
n_labels = len(df_data_merged_rnd)
mu_param = 10^-2

In [32]:
data.shape

(1, 200)

In [ ]:
# x_ini = np.zeros([agents,attributes])
def argmin(x_block,x,y):
    ### minimizer for x_tilda
    
def r0():
    
    
def smooth(x,m): ### Incorrect m(agents)
    lamda = 0.15 #given
    r = lamda* sum(r0(x[j]) for j in range(m))
    

def del_x(i,j,x,x_big,y_big,l,agents,pass_value): # x is the block of a single agent
    if i == j & pass_value == False:
        delta_x = np.zeros([x.shape[0]])
    else:
        x_tilda = argmin(x,x_big,y_big) + smooth(x,agents)
        delta_x = x_tilda - x #should be an array
    return delta_x

def graph_connection(i, agents, A_matrix):
    return agents[np.where(A_matrix[i,:] != 0)]
    
def graph_passing(i, agents, l, l_agent_new, Ni):
    if len(np.where(l_agent_new = l)) != 0:
        nit = Ni.intersection(agents[np.where(l_agent_new = l)])
    else:
        nit = np.array([])
    if len(nilt) == 0:
        nilt = i
        pass_value = False
    else:
        nilt = nit
        pass_value = True
    return nilt, pass_value

def func_a(i, j, A_matrix, pass_value):
    if j == i & pass_value == False:    
        a = 1
    else:
        a = A_matrix[i,j]
    return a

def block_sonata(dic_train_set, attributes, gamma, blocks, agents, max_iter, A_matrix):
    phi_now = np.ones([agents,blocks])
    phi_next = np.ones([agents,blocks])
    x_now = np.ones([agents,attributes])
    x_next = np.ones([agents,attributes])
    y_now = np.ones([agents,attributes])
    y_next = np.ones([agents,attributes])
    grad_now = np.ones([agents,attributes])
    grad_next = np.ones([agents,attributes])
    length = int(attributes/blocks)
    l_agent_new = random.randint(0,blocks,agents)
    f_values = np.zeros(epoch_size+1)
    for k in range(max_iter):
        l_agent = l_agent_new
        l_agent_new = random.randint(0,blocks,agents)
        gamma_new = gamma ** (k+1)
        for i in range(agents):
            Ni = graph_connection(i, agents, A_matrix)
            Nilt, pass_value = graph_passing(i, agents, l, l_agent_new, Ni)
            for l in range(blocks):
                x_now_block = x_now[i,l*length:(l+1)*length]

                phi_next[i,l] = sum(func_a(i, j, A_matrix, pass_value)*phi_now[j,l] for j in range(Nilt))
                x_now_block = x_now[j,l*length:(l+1)*length]
                x_next_block = sum((func_a(i, j, A_matrix, pass_value)*phi_now[j,l]*
                                    (x_now[j,l*length:(l+1)*length]+gamma_new*del_x(i,j,x_now_block, x_now[j,:],y_now[j,:],l,agents,pass_value))
                                            for j in range(Nilt))/phi_next[i,l]       
                x_next[i,l*length:(l+1)*length] = x_next_block
                                   
                y_now_block= y_now[i,l*length:(l+1)*length]
                grad_next_block= grad_func(grad_now[i,l*length:(l+1)*length],l,l_agent)
                y_next_block = sum((func_a(i,j,l,l_agent,Ni)) 
                                   *(phi_now[j,l]*y_now[j,l*length:(l+1)*length]) for j in range(Nilt))/phi_next[i,l] 
                                   +((grad_next_block-grad_now_block)/phi_next[i,l])
                            
                x_next[i,l*length:(l+1)*length] = x_next_block
                y_next[i,l*length:(l+1)*length] = y_next_block
                grad_next[i,l*length:(l+1)*length] = grad_next_block
        if max_iter==0 or (k % ceil(max_iter/epoch_size)) == 0:    
            f_values[epoch_index] = f_obj(x_now)
            epoch_index += 1
                                   
        x_now=x_next      
        y_now=y_next
        phi_now=phi_next     
        grad_now=grad_next
        
        return x_now,f_values

In [ ]:
# Implementation for training the optimization model
attributes = len(dic_train_set[0].columns)-1
labels = len(dic_train_set[0])
mu = 10**(-3)
epoch_size = 5
max_iter = n_labels*epoch_size
step_size = 0.3
lamda = 0.15
theta = 0.7

X_sol_BS,f_vals_BS = block_sonata(dic_train_set, attributes, gamma, blocks, agents, max_iter, A_matrix)
    
fig = plt.figure(figsize=(8,6))


plt.plot(range(0,max_iter+1,ceil(max_iter/epoch_size)),f_vals_BS.tolist(),color='black',
         marker='v',markersize=10,linestyle='solid',label="Block-sonata",linewidth=4)

plt.legend(loc=3,fontsize=12)
plt.xlabel('Number of single gradient evaluations', color='#1C2833',fontsize=12)
plt.ylabel("Objective function value", color='#1C2833',fontsize=18)


plt.grid(True)

In [12]:
len(np.array([]))

0

In [4]:
x= [4,5,6,7,8,9]

In [5]:
x

[4, 5, 6, 7, 8, 9]

In [92]:
li=np.zeros([1,1])

In [94]:
li = 45

In [96]:
li.shape

AttributeError: 'int' object has no attribute 'shape'

In [77]:
c = phi[2][4:8]

In [78]:
c

array([0., 0., 0., 0.])

In [89]:
for i in range(20):
    print(random.randint(0,10))

5
10
5
8
2
0
10
0
9
4
7
8
4
1
6
2
6
2
8
1
